| | |
|:----------|:----------|
| Name      | Bayesian Modeling Workshop |
| Notebook  | Intro to PyMC |
| Author    | Joshuah Touyz, PhD |
| Version   | 0.1 |
|Last update| 02/29/20 |

## Exploring functionality of PyMC3

In [30]:
# - Loading libraries - 
import pymc3 as pm
import plotnine as pn
import pandas as pd
import numpy as np

### How do we specify a model?
#### Approach 1: using `with`
- At this point we are simply defining the computational graph
- Note that nothing has been run yet, we are simply setting up the structure to be able to run our samples

In [20]:
# - Model Context Objeft - 
with pm.Model() as model:
    parameter = pm.Exponential("poisson_param", 1/10.0)
    data_generator = pm.Poisson("data_generator", parameter)

#### Approach 2: Alternate way to specify a model

In [21]:
model = pm.Model() #<- instantiate first

with model:
    parameter = pm.Exponential("poisson_param", 1/10.0)
    data_generator = pm.Poisson("data_generator", parameter)

#### How do we transform our variables?
- Transforming variables is simpled we simply add them together

In [22]:
# - Deterministic transform - 
with model:
    data_plus_one = data_generator + 1

#### How do we check the parameter assignments are working?
- PyMC3 has stored test_values (these are simply the default) values
- We can call them to make sure we are in fact getting the correct values

In [23]:
# Initial test value    
print("parameter.tag.test_value =", parameter.tag.test_value)
print("data_generator.tag.test_value =", data_generator.tag.test_value)
print("data_plus_one.tag.test_value =", data_plus_one.tag.test_value)

parameter.tag.test_value = 6.931471824645995
data_generator.tag.test_value = 6
data_plus_one.tag.test_value = 7


### How do I generate new values?
- To generate a sample you need to tell `pymc3` to sample from the distribution using `pm.sample'
- In the `pm.sample` you also specify the number `chains` and the `step` (sampler)
- You can also specify starting values that replace the intial test_value by `pm.Poisson("data_generator", parameter, testval = 0.5)`

In [24]:
# - Sampling from the distribution - 
with model:
    step = pm.Metropolis()
    trace = pm.sample(draws = 50, chains = 2, step = step)

Only 50 samples in chain.
Multiprocess sampling (2 chains in 4 jobs)
CompoundStep
>Metropolis: [data_generator]
>Metropolis: [poisson_param]
Sampling 2 chains, 0 divergences: 100%|██████████| 1100/1100 [00:00<00:00, 7691.61draws/s]
/Users/joshuahtouyz/Desktop/projects/project_1/p1_venv/lib/python3.7/site-packages/pymc3/sampling.py:496: UserWarning: The number of samples is too small to check convergence reliably.
  warnings.warn("The number of samples is too small to check convergence reliably.")


#### How do I check the simulated values?
- You will need to call the `trace` function and the name of the variable

In [25]:
simulated_data = pd.DataFrame({'exp_simulated':trace['poisson_param'] , 
                               'poisson_simullated':trace['data_generator'] 
                              }).unstack().reset_index()
simulated_data.columns= ['parameter','simulation','value']
simulated_data

,parameter,simulation,value
0,exp_simulated,0,8.607798
1,exp_simulated,1,8.938526
2,exp_simulated,2,8.938526
3,exp_simulated,3,8.938526
4,exp_simulated,4,8.938526
...,...,...,...
195,poisson_simullated,95,8.000000
196,poisson_simullated,96,5.000000
197,poisson_simullated,97,5.000000
198,poisson_simullated,98,13.000000


#### How do I overwrite a model?
- Overwriting is simple we re-instantiate the model with the same name
- Note that we can also add to a particular model provided it has not yet been run using a sampler

In [26]:
#- Adding to the model and overwriting data generator - 
with pm.Model() as model:
    theta = pm.Exponential("theta", 2.0)
    data_generator = pm.Poisson("data_generator", theta)
    

#### How do I define a new model?

In [27]:
# - Instantiating a new model - 
with pm.Model() as ab_testing:
    p_A = pm.Uniform("P(A)", 0, 1)
    p_B = pm.Uniform("P(B)", 0, 1)

#### What type of samplers does PyMC3 have?
- The `step` method is used to specify the MCMC sampler for generating posterior samples
- The default step methods will depend on the output variable:
    - Binary variables will be assigned to BinaryMetropolis
    - Discrete variables will be assigned to Metropolis
    - Continuous variables will be assigned to NUTS
- To conduct MCMC sampling to generate posterior samples in PyMC3, we specify a step method object that corresponds to a 

In [28]:
# Set of different samplers:
list(filter(lambda x: x[0].isupper(), dir(pm.step_methods)))

['BinaryGibbsMetropolis',
 'BinaryMetropolis',
 'CategoricalGibbsMetropolis',
 'CauchyProposal',
 'CompoundStep',
 'DEMetropolis',
 'ElemwiseCategorical',
 'EllipticalSlice',
 'HamiltonianMC',
 'LaplaceProposal',
 'Metropolis',
 'MultivariateNormalProposal',
 'NUTS',
 'NormalProposal',
 'PoissonProposal',
 'Slice']

#### How do I specify the sampler in my model?
- The `step` argument allows you to specify the sampler you intend to use
- Notice in the example below that you can specifiy different types of smapler for different parameter variables

In [31]:
with pm.Model() as model:
    mu = pm.Normal('mu', mu=0, sigma=1)
    sd = pm.HalfNormal('sd', sigma=1)
    obs = pm.Normal('obs', mu=mu, sigma=sd, observed=np.random.randn(100))

    step1 = pm.Metropolis(vars=[mu])
    step2 = pm.Slice(vars=[sd])
    trace = pm.sample(10000, step=[step1, step2], cores=4)

Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [mu]
>Slice: [sd]
Sampling 4 chains, 0 divergences: 100%|██████████| 42000/42000 [00:05<00:00, 8180.37draws/s]
The number of effective samples is smaller than 25% for some parameters.


#### How do I see the summary views of my sampler?

In [33]:
pm.summary(trace)

,mean,sd,hpd_3%,hpd_97%,mcse_mean,mcse_sd,ess_mean,ess_sd,ess_bulk,ess_tail,r_hat
mu,-0.048,0.108,-0.250,0.149,0.001,0.001,5361.0,4859.0,5371.0,5424.0,1.0
sd,1.066,0.077,0.923,1.208,0.000,0.000,36303.0,35670.0,37241.0,28880.0,1.0


#### How do I find the MAP estimate for my sample?
- It is frequently best practice to initialize the sampler with the map_estimate to speed up convergence

In [52]:
# Getting the MAP
map_estimate = pm.find_MAP(model=model)
print('The map estimate for mu is {mu} and sd is {sd}'.format(mu= np.round(map_estimate['mu']), sd = np.round(map_estimate['sd'])))

logp = -158.13, ||grad|| = 74.488: 100%|██████████| 9/9 [00:00<00:00, 2558.89it/s]

The map estimate for mu is -0.0 and sd is 1.0


#### How do I incorporate data into my models?
- So far most of the xample are those of unrealized random variables
- For an observed random variable we use `observed`
- Suppose we have the model:
$$
X|\mu,\tau^2 \sim N(\mu,\tau^2)\qquad \mu\sim Exp(1/10), \tau^2\sim HalfCauchy(scale = 5)
$$

In [63]:
# Random data centered around 5 with sd = 1
data_in = np.random.normal(size = 1000, loc = 5)

# - Model Context Objeft - 
with pm.Model() as model:
    prior_mu = pm.Exponential('mu', 1/10.0)
    prior_sd = pm.HalfCauchy('sd', beta = 5)
    
    # Inlcuding observed data
    lik = pm.Normal('lik', mu=prior_mu, sigma=prior_sd, observed=data_in)

    trace = pm.sample(10000, cores=2)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [sd, mu]
Sampling 2 chains, 0 divergences: 100%|██████████| 21000/21000 [00:05<00:00, 3575.97draws/s]
The acceptance probability does not match the target. It is 0.8838442646150758, but should be close to 0.8. Try to increase the number of tuning steps.


In [64]:
pm.summary(trace)

,mean,sd,hpd_3%,hpd_97%,mcse_mean,mcse_sd,ess_mean,ess_sd,ess_bulk,ess_tail,r_hat
mu,5.022,0.032,4.964,5.082,0.0,0.0,17970.0,17970.0,17931.0,13866.0,1.0
sd,0.985,0.022,0.943,1.025,0.0,0.0,17641.0,17604.0,17684.0,12980.0,1.0


#### How do I get help with the distributions
- Use the `help(pm.Normal)` where normal is replaced by one of the available distributions
- See here for ehe

#### How do I set the number of cores, samples, burn-in and chains?
- All of that goes into the `pm.sample` argumetn

In [68]:
with pm.Model() as model:
    mu = pm.Normal('mu', mu=0, sigma=1)
    obs = pm.Normal('obs', mu=mu, sigma=1, observed=np.random.randn(100))

    trace = pm.sample(cores=4,
                     draws=1000,
                      step=None, # <- specify sampler here
                      chains=4,
                      tune=500 # <- you can think of this as burn-in
                     )
print('The length of the chains is: {}'.format(len(trace)))

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [mu]
Sampling 4 chains, 0 divergences: 100%|██████████| 4800/4800 [00:00<00:00, 7680.45draws/s]
The acceptance probability does not match the target. It is 0.9181221704206914, but should be close to 0.8. Try to increase the number of tuning steps.
The acceptance probability does not match the target. It is 0.9232550505232099, but should be close to 0.8. Try to increase the number of tuning steps.
The acceptance probability does not match the target. It is 0.9172552597197268, but should be close to 0.8. Try to increase the number of tuning steps.
The acceptance probability does not match the target. It is 0.9271431676059172, but should be close to 0.8. Try to increase the number of tuning steps.


The length of the chains is: 1000


#### What else should I know?
- To speed up computation PyMC3 uses a couple of tricks, including transforming the underlying variables into a space before rescaling them back into normal space

#### What if I need a quick and dirty model?
- PyMC3 has variational methods 
- They are generally much quicker however they are less accurate and lead to biased estimates
- The analysis proceeds much in the same way, the main difference is with respect to how the samples are generated - instead of `pm.sample` we use `pm.fit`

In [70]:
with pm.Model() as model:
    mu = pm.Normal('mu', mu=0, sigma=1)
    sd = pm.HalfNormal('sd', sigma=1)
    obs = pm.Normal('obs', mu=mu, sigma=sd, observed=np.random.randn(100))

    approx = pm.fit()

Average Loss = 150.65: 100%|██████████| 10000/10000 [00:02<00:00, 3946.01it/s]
Finished [100%]: Average Loss = 150.65


#### How do I predict on out of sample data?
- There are two methods:
    - Include NAs for your target variable in your input data set and they will be estimated
    - Work with Theano's shared memory (this is a bit more invovled)

In [71]:
# Import theano and set up shared variables
import theano

x = np.random.randn(100)
y = x > 0

x_shared = theano.shared(x)
y_shared = theano.shared(y)

with pm.Model() as model:
    coeff = pm.Normal('x', mu=0, sigma=1)
    logistic = pm.math.sigmoid(coeff * x_shared)
    pm.Bernoulli('obs', p=logistic, observed=y_shared)
    trace = pm.sample()

# Updating set of variables of interest 
x_shared.set_value([-1, 0, 1.])
y_shared.set_value([0, 0, 0]) # dummy values

with model:
    post_pred = pm.sample_posterior_predictive(trace, samples=500)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [x]
Sampling 4 chains, 0 divergences: 100%|██████████| 4000/4000 [00:00<00:00, 7013.27draws/s]
The acceptance probability does not match the target. It is 0.8809534998885024, but should be close to 0.8. Try to increase the number of tuning steps.
/Users/joshuahtouyz/Desktop/projects/project_1/p1_venv/lib/python3.7/site-packages/pymc3/sampling.py:1247: UserWarning: samples parameter is smaller than nchains times ndraws, some draws and/or chains may not be represented in the returned posterior predictive sample
  "samples parameter is smaller than nchains times ndraws, some draws "
100%|██████████| 500/500 [00:03<00:00, 135.85it/s]


In [72]:
# Generate the posterior predictive
post_pred['obs'].mean(axis=0)


array([0.03 , 0.47 , 0.972])